In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/LongRange3D/')
from models_binnedFeat import RandomForest, SVC
from metrics import ClassificationResult
from utils import concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale 

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN X (CNMeM is enabled with initial size: 2500 MB, cuDNN 5005)


In [2]:
contacts='PE'
path='/users/mtaranov/datasets_3d_OLD/dist_matched_'+contacts+'/'

X_train = np.load(path+'motifs/train_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_test = np.load(path+'motifs/test_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')
X_valid = np.load(path+'motifs/valid_'+contacts+'_out_btw_nodes_3_0.0001/mat.npy')

y_train = get_labels(path+'y_train_thres_10.npy')
y_test = get_labels(path+'y_test_thres_10.npy')
y_valid = get_labels(path+'y_valid_thres_10.npy')


In [3]:
def normalize_features(X_train, X_valid, X_test, normalizer=StandardScaler):
    # fit normalizer
    normalizer = normalizer().fit(X_train[:,:])
    # transform features
    X_train_new=copy.copy(X_train)
    X_valid_new=copy.copy(X_valid)
    X_test_new=copy.copy(X_test)
    X_train_new = normalizer.transform(X_train)
    X_valid_new = normalizer.transform(X_valid)
    X_test_new = normalizer.transform(X_test)

    return (X_train_new, X_valid_new, X_test_new)

In [4]:
X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)

# Random Forest

In [5]:
rf = RandomForest()

### with motifs only

In [6]:
rf.train(X_train_normalized, y_train)
preds_test = rf.predict(X_test_normalized)
preds_train = rf.predict(X_train_normalized)
print ('On Test:\n{}\n'.format(ClassificationResult(y_test, preds_test)))
print ('On Train:\n{}\n'.format(ClassificationResult(y_train, preds_train)))

/users/mtaranov/LongRange3D/models_binnedFeat.py:231: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.classifier.fit(X, y)


On Test:
Balanced Accuracy: 85.10%	 auROC: 0.930	 auPRC: 0.944	 auPRG: 0.904
Recall at 5%|10%|20% FDR: 73.2%|80.0%|88.9%	 Num Positives: 906	 Num Negatives: 906	 

On Train:
Balanced Accuracy: 100.00%	 auROC: 1.000	 auPRC: 1.000	 auPRG: 1.000
Recall at 5%|10%|20% FDR: 100.0%|100.0%|100.0%	 Num Positives: 2412	 Num Negatives: 2412	 



In [32]:
scores=rf.ImportanceSelect()
#scores=scores.reshape(scores.shape[0],1)

In [33]:
scores

array([ 0.00076454,  0.00076425,  0.00059805, ...,  0.00045475,
        0.00049064,  0.00027912])

In [35]:
motif_names=np.loadtxt('/users/mtaranov/datasets_3d/dist_matched_PE/motifs/train_PE_out_btw_nodes_3_0.0001/motif_names.txt', dtype=str, delimiter='/')[:,6:].reshape(640,)
top_3_hits_names=[]
for i in motif_names:
    for j in range(3):
        top_3_hits_names.append(i)
top_3_hits_names = np.array(top_3_hits_names)

In [42]:
RF_Importance_Window_Motifs=top_3_hits_names[scores.argsort()[::-1]]

In [43]:
print np.where(RF_Importance_Window_Motifs=='CTCF_HUMAN')
print np.where(RF_Importance_Window_Motifs=='CTCFL_HUMAN')

(array([0, 3, 5]),)
(array([1, 2, 4]),)


In [46]:
RF_Importance_Window_Motifs[:100]

array(['CTCF_HUMAN', 'CTCFL_HUMAN', 'CTCFL_HUMAN', 'CTCF_HUMAN',
       'CTCFL_HUMAN', 'CTCF_HUMAN', 'NRF1_HUMAN', 'NRF1_HUMAN',
       'P73_HUMAN', 'NRF1_HUMAN', 'SP2_HUMAN', 'ZFX_HUMAN', 'FOXL1_HUMAN',
       'THAP1_HUMAN', 'NDF1_HUMAN', 'SP2_HUMAN', 'KLF15_HUMAN',
       'ALX1_HUMAN', 'P73_HUMAN', 'ZKSC1_HUMAN', 'ARNT2_HUMAN',
       'HES1_HUMAN', 'KLF14_HUMAN', 'EGR1_HUMAN', 'ZKSC1_HUMAN',
       'ZN350_HUMAN', 'RXRG_HUMAN', 'HME1_HUMAN', 'FOXO6_HUMAN',
       'ASCL2_HUMAN', 'EGR1_HUMAN', 'THAP1_HUMAN', 'ZFX_HUMAN',
       'HNF4G_HUMAN', 'NFIA_HUMAN', 'NDF1_HUMAN', 'WT1_HUMAN',
       'TBX3_HUMAN', 'EGR2_HUMAN', 'ZKSC1_HUMAN', 'TBX1_HUMAN',
       'ZN639_HUMAN', 'LHX9_HUMAN', 'THAP1_HUMAN', 'SP2_HUMAN',
       'MYBA_HUMAN', 'AP2A_HUMAN', 'TLX1_HUMAN', 'HEN1_HUMAN', 'SP4_HUMAN',
       'ETV7_HUMAN', 'SP4_HUMAN', 'E2F3_HUMAN', 'NR1I2_HUMAN',
       'NR2C2_HUMAN', 'ARNT2_HUMAN', 'IRX2_HUMAN', 'TF7L2_HUMAN',
       'SP4_HUMAN', 'PAX5_HUMAN', 'HES1_HUMAN', 'SRF_HUMAN', 'KLF15_HUMAN',
  